# Anostep cumulative survey as completed by each CHP (update daily)

Here are some quick analyses to look at the data so far :)
We start by accessing the data and cleaning it up

In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import json
import os
import webbrowser
from datetime import datetime, timedelta

def load_cleaned_data(filename="commcare_cleaned_data.csv"):
    """Load the cleaned data from CSV file"""
    try:
        if not os.path.exists(filename):
            print(f"File '{filename}' not found!")
            print("Please run the data loader script first.")
            return None
        
        df = pd.read_csv(filename)
        return df
    
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

def find_columns(df, column_type):
    """Find columns based on type"""
    if column_type == 'username':
        return [col for col in df.columns if 'username' in col.lower()]
    elif column_type == 'anoph':
        return [col for col in df.columns if 'anoph' in col.lower()]
    elif column_type == 'county':
        return [col for col in df.columns if 'county' in col.lower()]
    else:
        return []


def create_interactive_username_graph(df):
    """Create interactive username count graph with functional dropdown county filter"""
    
    # Find username, anoph, and county columns
    username_cols = find_columns(df, 'username')
    anoph_cols = find_columns(df, 'anoph')
    county_cols = find_columns(df, 'county')
    
    if not username_cols:
        print("No username column found")
        return None
    
    username_col = username_cols[0]
    anoph_col = anoph_cols[0] if anoph_cols else None
    county_col = county_cols[0] if county_cols else None
    
    print(f"Using columns - Username: {username_col}")
    if anoph_col:
        print(f"Anoph: {anoph_col}")
    if county_col:
        print(f"County: {county_col}")
    
    # Filter out 'an_steph_test' and remove rows with missing usernames
    df_filtered = df[df[username_col] != 'an_steph_test'].copy()
    df_filtered = df_filtered.dropna(subset=[username_col])
    df_filtered = df_filtered[df_filtered[username_col] != '']
    
    if df_filtered.empty:
        print("No data found after filtering")
        return None
    
    # Get unique counties
    unique_counties = []
    if county_col:
        unique_counties = sorted(df_filtered[county_col].dropna().unique())
    
    # Define colors for different anoph values
    color_dict = {}
    if anoph_col:
        anoph_values = df_filtered[anoph_col].unique()
        for col in anoph_values:
            if str(col).lower() == 'yes':
                color_dict[col] = '#2E8B57'  # Sea Green
            elif str(col).lower() == 'no':
                color_dict[col] = '#DC143C'  # Crimson
            elif str(col) == '___' or str(col) == '---' or str(col).strip() == '' or pd.isna(col):
                color_dict[col] = '#8B4513'  # Saddle Brown
            else:
                color_dict[col] = '#708090'  # Slate Gray

    def create_traces_for_county(county_filter=None):
        """Create traces for specific county or all counties"""
        if county_filter and county_col and county_filter != 'all':
            df_county = df_filtered[df_filtered[county_col] == county_filter]
        else:
            df_county = df_filtered
        
        if df_county.empty:
            return [], 0
        
        # Get username counts and sort alphabetically
        df_county = df_county.copy()
        df_county.loc[:, username_col] = df_county[username_col].astype(str)
        username_counts = df_county[username_col].value_counts()
        username_counts = username_counts.reindex(sorted(username_counts.index, key=str))
        
        # Create county mapping for each username
        county_mapping = {}
        if county_col:
            for username in username_counts.index:
                user_counties = df_county[df_county[username_col] == username][county_col].dropna()
                if not user_counties.empty:
                    county_mapping[username] = user_counties.mode().iloc[0] if len(user_counties.mode()) > 0 else user_counties.iloc[0]
                else:
                    county_mapping[username] = "Unknown"
        
        traces = []
        
        if anoph_col and anoph_col in df_county.columns:
            # Create stacked bar chart colored by anoph_present
            crosstab = pd.crosstab(df_county[username_col], df_county[anoph_col], dropna=False)
            sorted_index = sorted(crosstab.index, key=str)
            crosstab = crosstab.reindex(sorted_index, fill_value=0)
            
            for anoph_value in crosstab.columns:
                values = crosstab[anoph_value].values
                
                # Create hover text
                hover_text = []
                for i, (site, count) in enumerate(zip(crosstab.index, values)):
                    if count > 0:
                        total_for_site = crosstab.loc[site].sum()
                        percentage = (count / total_for_site) * 100 if total_for_site > 0 else 0
                        county_name = county_mapping.get(site, "Unknown") if county_mapping else "N/A"
                        hover_text.append(
                            f"<b>{site}</b><br>" +
                            f"County: {county_name}<br>" +
                            f"Anoph Present: {anoph_value}<br>" +
                            f"Count: {count}<br>" +
                            f"Percentage: {percentage:.1f}%<br>" +
                            f"Total for site: {total_for_site}"
                        )
                    else:
                        hover_text.append("")
                
                traces.append(go.Bar(
                    name=f'Anoph: {anoph_value}',
                    x=crosstab.index,
                    y=values,
                    marker_color=color_dict[anoph_value],
                    marker_line=dict(width=0.5, color='white'),
                    hovertemplate='%{hovertext}<extra></extra>',
                    hovertext=hover_text,
                    opacity=0.8,
                    visible=True
                ))
        else:
            # Simple bar chart if no anoph data
            hover_text = []
            for site, count in zip(username_counts.index, username_counts.values):
                county_name = county_mapping.get(site, "Unknown") if county_mapping else "N/A"
                hover_text.append(
                    f"<b>{site}</b><br>" +
                    f"County: {county_name}<br>" +
                    f"Total Surveys: {count}"
                )
            
            traces.append(go.Bar(
                x=username_counts.index,
                y=username_counts.values,
                marker_color='steelblue',
                marker_line=dict(width=0.5, color='navy'),
                hovertemplate='%{hovertext}<extra></extra>',
                hovertext=hover_text,
                opacity=0.8,
                visible=True
            ))
        
        return traces, len(df_county)

    # Create figure with initial data (all counties)
    fig = go.Figure()
    
    # Create separate traces for each county and "all"
    all_data = {}
    all_traces, all_count = create_traces_for_county()
    all_data['all'] = {'traces': all_traces, 'count': all_count}
    
    county_data = {}
    for county in unique_counties:
        county_traces, county_count = create_traces_for_county(county)
        if county_count > 0:
            county_data[county] = {'traces': county_traces, 'count': county_count}
    
    # Add all traces to figure (initially all visible, others hidden)
    trace_index = 0
    all_trace_indices = list(range(len(all_traces)))
    
    # Add "all" traces
    for trace in all_traces:
        trace.visible = True
        fig.add_trace(trace)
    trace_index += len(all_traces)
    
    # Add county-specific traces (initially hidden)
    county_trace_indices = {}
    for county, data in county_data.items():
        county_trace_indices[county] = list(range(trace_index, trace_index + len(data['traces'])))
        for trace in data['traces']:
            trace.visible = False
            fig.add_trace(trace)
        trace_index += len(data['traces'])
    
    # Create dropdown menu options
    dropdown_buttons = []
    
    # "All Counties" option
    visibility_all = [False] * len(fig.data)
    for idx in all_trace_indices:
        visibility_all[idx] = True
    
    dropdown_buttons.append(
        dict(
            label=f"All Counties ({all_count:,} surveys)",
            method="update",
            args=[
                {"visible": visibility_all},
                {"title": "Survey Count by Collection Site<br><sub>All Counties</sub>"}
            ]
        )
    )
    
    # County-specific options
    for county, data in county_data.items():
        visibility_county = [False] * len(fig.data)
        for idx in county_trace_indices[county]:
            visibility_county[idx] = True
        
        dropdown_buttons.append(
            dict(
                label=f"{county} ({data['count']:,} surveys)",
                method="update",
                args=[
                    {"visible": visibility_county},
                    {"title": f"Survey Count by Collection Site<br><sub>Filtered by {county}</sub>"}
                ]
            )
        )
    
    # Update layout
    total_surveys = len(df_filtered)
    unique_sites = df_filtered[username_col].nunique()
    unique_counties_count = len(unique_counties) if unique_counties else 0
    
    title_text = f'Survey Count by Collection Site<br><sub>{unique_sites} sites across {unique_counties_count} counties</sub>'
    
    fig.update_layout(
        title={
            'text': title_text,
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16}
        },
        xaxis_title='Collection Site',
        yaxis_title='Count of Survey',
        barmode='stack' if anoph_col else 'group',
        hovermode='closest',
        width=1200,
        height=700,
        font=dict(size=12),
        showlegend=True if anoph_col else False,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1,
            xanchor="left",
            x=1.02
        ),
        margin=dict(l=80, r=120, t=120, b=80),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        # Add interactive dropdown menu
        updatemenus=[
            dict(
                buttons=dropdown_buttons,
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.02,
                xanchor="left",
                y=1.15,
                yanchor="top",
                bgcolor="rgba(255,255,255,0.9)",
                bordercolor="#007bff",
                borderwidth=2,
                font=dict(size=12)
            ),
        ]
    )
    
    # Update x-axis
    fig.update_xaxes(
        tickangle=45,
        tickfont=dict(size=10),
        gridcolor='lightgray',
        gridwidth=0.5,
        showgrid=True
    )
    
    # Update y-axis
    fig.update_yaxes(
        gridcolor='lightgray',
        gridwidth=0.5,
        showgrid=True,
        zeroline=True,
        zerolinecolor='gray',
        zerolinewidth=1
    )
    
    return fig, df_filtered, unique_counties

def create_summary_dashboard(df):
    """Create a comprehensive dashboard with multiple views"""
    
    username_cols = find_columns(df, 'username')
    anoph_cols = find_columns(df, 'anoph')
    
    if not username_cols:
        print("No username column found for dashboard")
        return None
    
    username_col = username_cols[0]
    anoph_col = anoph_cols[0] if anoph_cols else None
    
    # Filter data
    df_filtered = df[df[username_col] != 'an_steph_test'].copy()
    df_filtered = df_filtered.dropna(subset=[username_col])
    df_filtered = df_filtered[df_filtered[username_col] != '']
    
    if df_filtered.empty:
        return None
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Survey Count by Collection Site',
            'Anopheles Present Distribution',
            'Top 10 Most Active Sites',
            'Survey Activity Overview'
        ),
        specs=[[{"secondary_y": False}, {"type": "pie"}],
               [{"type": "bar"}, {"type": "indicator"}]],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    # 1. Main bar chart (top left)
    username_counts = df_filtered[username_col].value_counts()
    username_counts = username_counts.reindex(sorted(username_counts.index, key=str))
    
    if anoph_col and anoph_col in df_filtered.columns:
        crosstab = pd.crosstab(df_filtered[username_col], df_filtered[anoph_col], dropna=False)
        sorted_index = sorted(crosstab.index, key=str)
        crosstab = crosstab.reindex(sorted_index, fill_value=0)
        
        colors = ['#2E8B57', '#DC143C', '#8B4513', '#708090']
        for i, anoph_value in enumerate(crosstab.columns):
            color = colors[i % len(colors)]
            fig.add_trace(
                go.Bar(
                    name=f'Anoph: {anoph_value}',
                    x=crosstab.index,
                    y=crosstab[anoph_value].values,
                    marker_color=color,
                    showlegend=True
                ),
                row=1, col=1
            )
    
    # 2. Pie chart (top right)
    if anoph_col and anoph_col in df_filtered.columns:
        anoph_counts = df_filtered[anoph_col].value_counts()
        fig.add_trace(
            go.Pie(
                labels=anoph_counts.index,
                values=anoph_counts.values,
                marker_colors=['#2E8B57', '#DC143C', '#8B4513', '#708090'][:len(anoph_counts)],
                hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>'
            ),
            row=1, col=2
        )
    
    # 3. Top 10 sites (bottom left)
    top_10_sites = username_counts.head(10)
    fig.add_trace(
        go.Bar(
            x=top_10_sites.values,
            y=top_10_sites.index,
            orientation='h',
            marker_color='lightblue',
            marker_line=dict(width=1, color='navy'),
            showlegend=False,
            hovertemplate='<b>%{y}</b><br>Surveys: %{x}<extra></extra>'
        ),
        row=2, col=1
    )
    
    # 4. Summary indicators (bottom right)
    total_surveys = len(df_filtered)
    unique_sites = len(username_counts)
    avg_per_site = total_surveys / unique_sites if unique_sites > 0 else 0
    
    fig.add_trace(
        go.Indicator(
            mode="number+delta",
            value=total_surveys,
            title={"text": "Total Surveys"},
            domain={'x': [0, 0.5], 'y': [0.7, 1]},
            number={'font': {'size': 40}},
            delta={'reference': total_surveys * 0.9, 'relative': True}
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        title={
            'text': 'CommCare Survey Dashboard',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 20}
        },
        height=800,
        width=1400,
        showlegend=True,
        barmode='stack'
    )
    
    return fig

def main():
    try:
        # Load the cleaned data
        df = load_cleaned_data()
        if df is None:
            return
        
        print("Creating interactive survey visualization with functional county dropdown...")
        
        # Create interactive graph with county dropdown
        result = create_interactive_username_graph(df)
        
        if result is not None:
            if len(result) == 3:
                fig, df_filtered, unique_counties = result
            else:
                print("Unexpected result format")
                return
            
            # Save as HTML
            main_filename = 'interactive_survey.html'
            fig.write_html(
                main_filename,
                config={
                    'displayModeBar': True,
                    'displaylogo': False,
                    'modeBarButtonsToAdd': ['drawline', 'drawopenpath', 'drawclosedpath', 'drawcircle', 'drawrect', 'eraseshape']
                }
            )
            
            # print(f"Interactive survey count graph with functional dropdown saved as: {main_filename}")
            # if unique_counties:
            #     print(f"Counties included: {', '.join(unique_counties)}")
            
            # Open in browser
            try:
                print("Opening interactive visualization in browser...")
                main_path = os.path.abspath(main_filename)
                webbrowser.open(f'file://{main_path}')
                
            except Exception as e:
                print(f"Could not open browser automatically: {e}")
                print(f"Please manually open: {main_filename}")
            
            print("\nInteractive features:")
            print("- Use the dropdown menu in the top-left corner to filter by county")
            print("- Chart automatically updates when you select different counties")
            print("- Hover over bars for detailed information")
            print("- Click legend items to show/hide categories")
            print("- Use toolbar for zoom, pan, and selection")
            print("- Double-click to reset zoom")
            
        else:
            print("Failed to create interactive graph")
    
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Creating interactive survey visualization with functional county dropdown...
Using columns - Username: username
Anoph: anoph_present
County: county
Opening interactive visualization in browser...

Interactive features:
- Use the dropdown menu in the top-left corner to filter by county
- Chart automatically updates when you select different counties
- Hover over bars for detailed information
- Click legend items to show/hide categories
- Use toolbar for zoom, pan, and selection
- Double-click to reset zoom


### Great work team!👏 👏 👏